# Single Rendevous Experiments with GTOC5

## Sorting out imports

In [1]:
import pykep as pk
import numpy as np
from tqdm import tqdm, trange

In [2]:
import matplotlib.pylab as plt
%matplotlib inline
import seaborn as sns
plt.rcParams['figure.figsize'] = 10, 8

In [3]:
import src
import src.gtoc5
import src.gtoc5.gtoc5
from src.gtoc5.gtoc5 import *

loaded src module
Loaded gtoc5 submodule


# Basic steps for assembling GTOC5 trajectories

The two primary functions for assembling a GTOC5 trajectory are here `mission_to_1st_asteroid()` and `add_asteroid()`. The first initializes the mission's data structure with the details of the Earth launch leg, that takes the spacecraft towards the mission's first asteroid. Subsequently, via multiple calls to `add_asteroid()`, the mission is extended with additional exploration targets. Each call to `add_asteroid()` creates a rendezvous leg towards the specified asteroid, immediately followed by a flyby of that same asteroid, and so increases the mission's overall score by 1.

Here's an example of a mission that launches towards asteroid `1712`, and moves next to asteroid `4893`. The `True` value returned by `add_asteroid()` indicates that a feasible transfer leg was found, and asteroid `4893` was therefore added to the mission.

In [4]:
t = mission_to_1st_asteroid(1712)

(1712, 3912.530999146394, 59127.205255048466, 59181.08381655966)


In [5]:
add_asteroid(t, 4893)

True

We can evaluate this trajectory with respect to its score (number of asteroids fully explored), final mass (in kg), and time of flight (here converted from days to years).

In [6]:
score(t), final_mass(t), tof(t) * DAY2YEAR

(2.0, 3484.7515275785117, 1.615074770344698)

An aggregation of the mission's mass and time costs can be obtained with `resource_rating()`. It measures the extent to which the mass and time budgets available for the mission have been depleted by the trajectory. It produces a value of 1.0 at the start of the mission, and a value of 0.0 when the mission has exhausted its 3500 kg of available mass, or its maximum duration of 15 years.

In [7]:
resource_rating(t)

0.8721092603416899

As the score increments discretely by 1.0 with each added asteroid, and the resource rating evaluates mass and time available in a range of [0, 1], both can be combined to give a single-objective evaluation of the trajectory, that should be maximized:

In [8]:
score(t) + resource_rating(t)

2.87210926034169

Calling `seq()`, we can see either the full sequence of asteroids visited in each leg, or just the distinct asteroids visited in the mission. In this example, we see that the mission starts on Earth (id `0`), performs a rendezvous with asteroid `1712`, followed by a flyby of the same asteroid, and then repeats the pattern at asteroid `4893`.

In [9]:
print(seq(t))
print(seq(t, incl_flyby=False))

[0, 1712, 1712, 4893, 4893]
[0, 1712, 4893]


The trajectory data structure built by `mission_to_1st_asteroid()` and `add_asteroid()` is a list of tuples summarizing the evolution of the spacecraft's state. It provides the minimal sufficient information from which a more detailed view can be reproduced, if so desired. Each tuple contains:

1. asteroid ID
2. spacecraft mass
3. [epoch][epoch]
4. the leg's $\Delta T$
5. the leg's $\Delta V$

The mass and epoch values correspond to the state at the given asteroid, at the end of a rendezvous or self-fly-by leg, after deploying the corresponding payload. The $\Delta T$ and $\Delta V$ values refer to that leg that just ended.

[epoch]: https://en.wikipedia.org/wiki/Epoch_(astronomy)

In [10]:
t[-1]

(4893,
 3484.7515275785117,
 59717.11131491687,
 134.19998321371767,
 965.6854249492379)

Epochs are given here as [Modified Julian Dates (MJD)](https://en.wikipedia.org/wiki/Julian_day#Variants), and can be converted as:

In [11]:
pk.epoch(t[-1][2], 'mjd')

2022-May-18 02:40:17.608817

## Greedy search

In this section we perform a [Greedy search](https://en.wikipedia.org/wiki/Greedy_algorithm) for a GTOC5 trajectory. We'll start by going to asteroid `1712`. Then, and at every following step, we attempt to create legs towards all still available asteroids. Among equally-scored alternatives, we greedily pick the one with highest resource rating to adopt into the trajectory, and continue from there. Search stops when no feasible legs are found that can take us to another asteroid. This will happen either because no solutions were found that would allow for a leg to be created, or because adding a found solution would require the spacecraft to exceed the mission's mass or time budgets.

In [30]:
def greedy_step(traj):
    traj_asts = set(seq(traj, incl_flyby=False))
    extended = []
    for a in range(len(asteroids)):
        if a not in traj_asts: # only consider asteroids which have not been visited before
            tt = traj.copy()
            if add_asteroid(tt, next_ast=a, use_cache=False, obj_fun=lambert_eval):
                extended.append(tt)
    
    return max(extended, key=resource_rating, default=[])

In [31]:
t = mission_to_1st_asteroid(1712)
t = greedy_step(t)

(1712, 3912.530999146394, 59127.205255048466, 59181.08381655966)


AttributeError: 'int' object has no attribute 'eph'

In [28]:
print(seq(t, incl_flyby=False))

[0, 1712, 4893]


In [29]:
for l in t:
    print(type(l))
    print(l)

<class 'tuple'>
(0, 4000.0, 59127.205255048466, 0.0, 0.0)
<class 'tuple'>
(1712, 3872.530999146394, 59181.08381655966, 53.87856151119195, 650.4716282561097)
<class 'tuple'>
(1712, 3746.481928641157, 59325.360311294986, 144.27649473533114, 965.6854249492379)
<class 'tuple'>
(4893, 3602.066962005158, 59582.91133170315, 257.55102040816325, 831.5807789590036)
<class 'tuple'>
(4893, 3484.7515275785117, 59717.11131491687, 134.19998321371767, 965.6854249492379)
